In [1]:
import pandas as pd
import os
import numpy as np 

#Directory
os.chdir('C:\\Users\\Garrett\\Google Drive\\Documents\\Journal\\Surf Data Project')

#File path
filepath = 'C:\\Users\\Garrett\\Google Drive\\Documents\\Journal\\Surf Data Project\\data_raw\\Surf_Data_2020_12_30_v2.xlsx'

#Reading files
    #Surf Data Sheet:
df_2020 = pd.read_excel(filepath, sheet_name='2020')
df_2019 = pd.read_excel(filepath, sheet_name='2019')
df_2018 = pd.read_excel(filepath, sheet_name='2018')
df_2017 = pd.read_excel(filepath, sheet_name='2017')
df_coordinates = pd.read_excel(filepath, sheet_name='Coordinates')


In [20]:
#Putting them all together (Only Date, Spot, and Region are the same)
df_all = pd.concat([df_2017, df_2018, df_2019, df_2020], ignore_index=True)
#remove some columns in all b/c not needed in the entire dataset
df_all = df_all.drop(columns =
                     ['People', 'Texture', 'Wave Quality', 'Wave Height',
                      'Visuals', 'Notes', 'Surfing Quality', 'Sky',
                      'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 
                      'Unnamed: 20','BUOY Data', 'Hrs', 'Board', 'Wetty'])
#Make the date columns date
df_all['Date'] = pd.to_datetime(df_all[['Year', 'Month', 'Day']])
df_all['DateTime'] = df_all['Date'].dt.strftime('%Y/%m/%d %H:%M') #This is used for the kepler.gl animation

#Compile just the 2018-2020 data (Used for wave height)
savethesecolumns = ['Year', 'Month', 'Day', 'Wave Height', 'Wave Quality']
df_181920 = pd.concat([df_2018[savethesecolumns], 
                       df_2019[savethesecolumns], 
                       df_2020[savethesecolumns]], 
                      ignore_index=True)
df_181920['Date'] = pd.to_datetime(df_181920[['Year', 'Month', 'Day']])

#Change the wave height seperated by a comma into a single value (WH_avg)
df_181920 = pd.concat([df_181920[['Date', 'Year', 'Month', 'Day']], df_181920['Wave Height'].str.split(',', expand=True)], axis=1) #Split into two columns
df_181920['wh1'] = pd.to_numeric(df_181920.iloc[:, 4]) #Change object to int
df_181920['wh2'] = pd.to_numeric(df_181920.iloc[:, 5]) #Change object to int
df_181920['WH_avg'] = df_181920[['wh1', 'wh2']].mean(axis=1) #get the mean and assign it
df_181920 = df_181920.drop(columns = ['wh1', 'wh2', 0, 1]) #remove some columns
   
#Adding to the 2018/2019/2020 concat df so we can do an analysis of waves at a spot
df_181920['wv_ql'] = pd.concat([df_2018['Wave Quality'], 
                                df_2019['Wave Quality'], 
                                df_2020['Wave Quality']], ignore_index=True)
df_181920['Spot'] = pd.concat([df_2018['Spot'], 
                               df_2019['Spot'], 
                               df_2020['Spot']], ignore_index=True)
df_181920['Region'] = pd.concat([df_2018['Region'], 
                                 df_2019['Region'], 
                                 df_2020['Region']], ignore_index=True)
    
#Changing Directory before saving
os.chdir('C:\\Users\\Garrett\\Google Drive\\Documents\\Journal\\Surf Data Project\\data_exported')

#Save the data from 2018-2020
df_181920.to_csv('df_181920.csv', index=False, header=True)

#Changing Directory back
os.chdir('C:\\Users\\Garrett\\Google Drive\\Documents\\Journal\\Surf Data Project')

In [38]:
#ANALYSIS OF SPOT QUALITY/CONSISTENCY

df_spotavg = pd.DataFrame()
df_spotavg['wv_ql_avg'] = round(df_181920.groupby('Spot')['wv_ql'].mean(), 2) #Find mean of the spot and round the value
df_spotavg['spot_count'] = df_181920.groupby('Spot')['wv_ql'].count() # Add the counts

#Grouping by spot and intereating through each spot to find the amount of times wave_qual is above 9
above9 = []
for (spotname, group) in df_181920.groupby('Spot'):
    groupinlist = group.wv_ql.tolist() 
    counter = 0
    for j in groupinlist: #
        if j > 8.9:
            counter = counter + 1
    above9.append(counter)
df_spotavg['abv9_count'] = above9
df_spotavg['abv9_prop'] = round(df_spotavg['abv9_count'] / df_spotavg['spot_count'], 2)


df_spotavg = df_spotavg.drop(df_spotavg[df_spotavg['spot_count'] < 10].index) #Removes colums if the spot was only surfed x amount of times
# df_spotavg = df_spotavg.sort_values(by='wv_ql_avg', ascending = False) #sorts by the wave quality
# df_spotavg = df_spotavg.sort_values(by='abv9_count', ascending = False) #sorts by the wave quality
df_spotavg = df_spotavg.sort_values(by='abv9_prop', ascending = False) #sorts by the wave quality

In [39]:
#ANALYSIS OF REGION QUALITY/CONSISTENCY

df_regionavg = pd.DataFrame()
#Find mean of the region and round the value
df_regionavg['wv_ql_avg'] = round(df_181920.groupby('Region')['wv_ql'].mean(), 2) 
df_regionavg['region_count'] = df_181920.groupby('Region')['wv_ql'].count() # Add the counts

#Grouping by Region and intereating through each Region to find the amount of times wave_qual is above 9
regionabove9 = []
for (regionname, group) in df_181920.groupby('Region'):
    groupinlist = group.wv_ql.tolist() 
    counter = 0
    for j in groupinlist: #
        if j > 8.9:
            counter = counter + 1
    regionabove9.append(counter)
df_regionavg['abv9_count'] = regionabove9
df_regionavg['abv9_prop'] = round(df_regionavg['abv9_count'] / df_regionavg['region_count'], 2)


df_regionavg = df_regionavg.drop(df_regionavg[df_regionavg['region_count'] < 10].index) #Removes colums if the spot was only surfed x amount of times
# df_spotavg = df_spotavg.sort_values(by='wv_ql_avg', ascending = False) #sorts by the wave quality
# df_spotavg = df_spotavg.sort_values(by='abv9_count', ascending = False) #sorts by the wave quality
df_regionavg = df_regionavg.sort_values(by='abv9_prop', ascending = False) #sorts by the wave quality

In [4]:
#AMOUNT OF TIMES/HRS IN WATER

#Adding 2019/2020 concat df so we can do an analysis of waves at a spot
df_1920 = pd.DataFrame()
df_1920['Month'] = pd.concat([df_2019['Month'], df_2020['Month']], ignore_index=True)
df_1920['Year'] = pd.concat([df_2019['Year'], df_2020['Year']], ignore_index=True)
df_1920['Hrs'] = pd.concat([df_2019['Wave Quality'], df_2020['Wave Quality']], ignore_index=True)

#This functions input the year and outputs the monthly average bins for that year
def month_sum(year):
    df = pd.DataFrame()
    df['hrs'] = df_1920.loc[df_1920['Year'] == year, 'Hrs']
    df['month'] = df_1920.loc[df_1920['Year'] == year, 'Month']
    df_output = pd.DataFrame()
    df_output['hrs_sum'] = df.groupby('month')['hrs'].sum()   
    df_output['count'] = df.groupby('month')['hrs'].count()
    df_output['year'] = year
    df_output['month'] = np.arange(1, 13)
    return df_output

#calling the function above and creating a new dataframe with the month, year, and month avg
df_monthly_time = pd.concat([month_sum(2019), month_sum(2020)], ignore_index=True)
df_monthly_time = round(df_monthly_time, 2)

df_monthly_time

,hrs_sum,count,year,month
0,198.0,28,2019,1
1,84.0,14,2019,2
2,154.5,21,2019,3
3,100.0,16,2019,4
4,51.5,9,2019,5
5,108.5,16,2019,6
6,148.0,25,2019,7
7,117.0,21,2019,8
8,43.0,7,2019,9
9,109.5,17,2019,10


In [21]:
#Merge the coordinates df with the spots df to put lat long on every spot sequentialy
df_test = pd.merge(df_all[['Spot']], df_coordinates, on='Spot', how='left')

#Get Data Ready For geospatial Animation
df_animation = pd.DataFrame()
df_animation['DateTime'] = df_all['DateTime']
df_animation['Spot'] = df_all['Spot']
df_animation['Latitude'] = df_test['Lat']
df_animation['Longitude'] = df_test['Long']
df_animation['Value'] = 10

#Changing Directory before saving
os.chdir('C:\\Users\\Garrett\\Google Drive\\Documents\\Journal\\Surf Data Project\\data_exported')

#Saving the data
df_animation.to_csv('animation_ready.csv', index=False)

#Changing directory after saving
os.chdir('C:\\Users\\Garrett\\Google Drive\\Documents\\Journal\\Surf Data Project')

In [ ]:
#Find unique spots within the coordinates sheet that do no corresond to the spots in the 'years' sheets
unique_coor_spots = np.unique(df_coordinates['Spot'])
unique_all_spots = np.unique(df_all['Spot'])

#Find spots which I surfed that I do not have coordinate attached to them
need_coor = []
for i in range(0, len(unique_all_spots)):
    if unique_all_spots[i] not in unique_coor_spots:
        print(unique_all_spots[i])
        need_coor.append(unique_all_spots[i])

In [ ]:
#Find Occurance of spots!
df_all_occur = df_all['Spot'].value_counts()

#Turn occurence into a dataframe
df_all_occur = pd.DataFrame(df_all_occur)
#Reset the index and change the names of the columns
df_all_occur = df_all_occur.reset_index()
df_all_occur.rename(columns = {'index': 'Spot', 'Spot': 'Occurrence'}, inplace = True) 

#Now merge the coordinates with occurance
df_all_coor_occur = pd.merge(df_coordinates, df_all_occur, how='inner', on = 'Spot')
df_all_coor_occur = df_all_coor_occur.drop(columns=['Unnamed: 3'])
df_all_coor_occur.rename(columns = {'Lat': 'Latitude', 'Long': 'Longitude'}, inplace = True) 

#Changing Directory befroe saving
os.chdir('C:\\Users\\Garrett\\Google Drive\\Documents\\Journal\\Surf Data Project\\data_exported')

#Saving the data
df_all_coor_occur.to_csv('Coor_with_Occur.csv', index=False)

#Changing directory after saving
os.chdir('C:\\Users\\Garrett\\Google Drive\\Documents\\Journal\\Surf Data Project')